Start with a toy dataset used in one of the first diffusion papers [Sohl-Dickstein et.al. 2015], where $$ \Kset \subset \R^2 $$ are points sampled from a spiral. We first construct and visualize this dataset

In [ ]:
dataset = Swissroll(np.pi/2, 5*np.pi, 100)
loader  = DataLoader(dataset, batch_size=2048)

For this simple dataset, we can implement the denoiser using a multi-layer perceptron (MLP):

In [ ]:
def get_sigma_embeds(sigma):
    sigma = sigma.unsqueeze(1)
    return torch.cat([torch.sin(torch.log(sigma)/2),
                      torch.cos(torch.log(sigma)/2)], dim=1)

class TimeInputMLP(nn.Module):
    def __init__(self, dim, hidden_dims):
        super().__init__()
        layers = []
        for in_dim, out_dim in pairwise((dim + 2,) + hidden_dims):
            layers.extend([nn.Linear(in_dim, out_dim), nn.GELU()])
        layers.append(nn.Linear(hidden_dims[-1], dim))
        self.net = nn.Sequential(*layers)
        self.input_dims = (dim,)

    def rand_input(self, batchsize):
        return torch.randn((batchsize,) + self.input_dims)

    def forward(self, x, sigma):
        sigma_embeds = get_sigma_embeds(sigma)         # shape: b x 2
        nn_input = torch.cat([x, sigma_embeds], dim=1) # shape: b x (dim + 2)
        return self.net(nn_input)

model = TimeInputMLP(dim=2, hidden_dims=(16,128,128,128,128,16))

The MLP takes the concatenation of $x \in \R^2$ and an embedding of the noise level \sigma, then predicts the noise $\epsilon \in \R^2$
. Although many diffusion models use a sinusoidal positional embedding for 
, the simple two-dimensional embedding works just as well: